# 🎨 ComfyUI 快速启动笔记本

此笔记本用于在 Google Colab Pro 上快速部署和启动 ComfyUI。

## 📝 使用说明

1. **首次使用**：运行所有单元格，会自动下载并部署环境（3-5分钟）
2. **后续使用**：直接运行所有单元格，会跳过下载步骤（<1分钟）
3. **更新环境**：删除 Drive 中的 ComfyUI 目录，重新运行即可

## ⚙️ 配置说明

在运行前，请修改下面的配置单元格中的 `SNAPSHOT_URL`

In [ ]:
# ==========================================
# 配置区域 - 请修改此处
# ==========================================

# 快照下载 URL（从 GitHub Release 获取）
SNAPSHOT_URL = "https://github.com/YOUR-USERNAME/comfyui-snapshot/releases/download/v20250101/comfyui_snapshot_20250101_120000.tar.gz"

# Google Drive 中的 ComfyUI 路径
DRIVE_COMFYUI_PATH = "/content/drive/MyDrive/ComfyUI"

# ComfyUI 启动端口
PORT = 8188

# GPU 模式（根据分配的 GPU 自动选择，一般不需要修改）
# 选项: 'auto', 'highvram', 'normalvram', 'lowvram', 'novram'
GPU_MODE = 'auto'

# 是否强制重新下载（即使已存在）
FORCE_DOWNLOAD = False

print("✅ 配置加载完成")
print(f"   快照 URL: {SNAPSHOT_URL}")
print(f"   Drive 路径: {DRIVE_COMFYUI_PATH}")

## 📂 步骤 1: 挂载 Google Drive

In [ ]:
from google.colab import drive
import os
import time

start_time = time.time()

print("📂 挂载 Google Drive...")
drive.mount('/content/drive')
print(f"✅ Drive 已挂载 ({time.time() - start_time:.1f}秒)\n")

## 🎮 步骤 2: 检查 GPU

In [ ]:
print("🎮 检查 GPU 信息...\n")
!nvidia-smi --query-gpu=name,memory.total,driver_version --format=csv

# 获取 GPU 名称
import subprocess
result = subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], 
                       capture_output=True, text=True)
gpu_name = result.stdout.strip()
print(f"\n✅ 检测到 GPU: {gpu_name}\n")

# 根据 GPU 自动选择启动模式
if GPU_MODE == 'auto':
    if 'A100' in gpu_name or 'H100' in gpu_name:
        LAUNCH_MODE = '--highvram'
        print("🚀 使用高性能模式 (highvram)")
    elif 'V100' in gpu_name:
        LAUNCH_MODE = ''
        print("🚀 使用标准模式")
    elif 'T4' in gpu_name:
        LAUNCH_MODE = '--lowvram'
        print("🚀 使用低显存模式 (lowvram)")
    else:
        LAUNCH_MODE = ''
        print("🚀 使用标准模式")
else:
    LAUNCH_MODE = f'--{GPU_MODE}' if GPU_MODE != 'normalvram' else ''
    print(f"🚀 使用指定模式: {GPU_MODE}")

## 📥 步骤 3: 下载和部署 ComfyUI

In [ ]:
import os
import time

step_start = time.time()

# 检查是否已存在
if os.path.exists(DRIVE_COMFYUI_PATH) and not FORCE_DOWNLOAD:
    print(f"✅ ComfyUI 环境已存在于: {DRIVE_COMFYUI_PATH}")
    print("   跳过下载步骤\n")
else:
    if FORCE_DOWNLOAD:
        print("🔄 强制重新下载模式\n")
    
    print("📥 下载 ComfyUI 快照...")
    print(f"   URL: {SNAPSHOT_URL}")
    print("   这可能需要 2-3 分钟，请稍候...\n")
    
    # 下载快照
    !wget -q --show-progress "{SNAPSHOT_URL}" -O /tmp/comfyui_snapshot.tar.gz
    
    if not os.path.exists('/tmp/comfyui_snapshot.tar.gz'):
        print("\n❌ 下载失败！请检查:")
        print("   1. URL 是否正确")
        print("   2. GitHub Release 是否公开")
        print("   3. 网络连接")
        raise Exception("快照下载失败")
    
    print("\n📦 解压快照到 Google Drive...")
    
    # 创建父目录
    parent_dir = os.path.dirname(DRIVE_COMFYUI_PATH)
    !mkdir -p "{parent_dir}"
    
    # 解压（tar 会自动创建 ComfyUI 目录）
    !tar -xzf /tmp/comfyui_snapshot.tar.gz -C "{parent_dir}"
    
    # 清理临时文件
    !rm /tmp/comfyui_snapshot.tar.gz
    
    print(f"\n✅ ComfyUI 已部署到: {DRIVE_COMFYUI_PATH}")
    print(f"   用时: {time.time() - step_start:.1f}秒\n")

## 📦 步骤 4: 安装依赖

In [ ]:
import os

step_start = time.time()

print("📦 安装 Python 依赖...\n")

# 切换到 ComfyUI 目录
%cd {DRIVE_COMFYUI_PATH}

# 安装 PyTorch（匹配 Colab 的 CUDA 版本）
print("安装 PyTorch (CUDA 12.1)...")
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# 安装 xformers
print("\n安装 xformers...")
!pip install -q xformers

# 如果有完整依赖清单，安装它
if os.path.exists('requirements_complete.txt'):
    print("\n安装完整依赖清单...")
    !pip install -q -r requirements_complete.txt
else:
    print("\n安装基础依赖...")
    !pip install -q -r requirements.txt

# 检查并安装自定义节点依赖
print("\n检查自定义节点依赖...")
if os.path.exists('custom_nodes'):
    for node_dir in os.listdir('custom_nodes'):
        node_path = os.path.join('custom_nodes', node_dir)
        req_file = os.path.join(node_path, 'requirements.txt')
        if os.path.isdir(node_path) and os.path.exists(req_file):
            print(f"  📦 安装 {node_dir} 的依赖...")
            !pip install -q -r "{req_file}"

print(f"\n✅ 依赖安装完成 ({time.time() - step_start:.1f}秒)\n")

## 🔗 步骤 5: 设置模型目录（可选）

In [ ]:
# 创建模型目录结构（如果不存在）
model_dirs = [
    'models/checkpoints',
    'models/vae',
    'models/loras',
    'models/controlnet',
    'models/animatediff_models',
    'models/animatediff_motion_lora',
    'models/upscale_models',
    'input',
    'output'
]

print("📁 检查目录结构...\n")
for dir_path in model_dirs:
    full_path = os.path.join(DRIVE_COMFYUI_PATH, dir_path)
    if not os.path.exists(full_path):
        os.makedirs(full_path, exist_ok=True)
        print(f"  ✓ 创建: {dir_path}")

print("\n✅ 目录结构检查完成")
print("\n💡 提示: 你可以手动上传模型到 Google Drive 的以下目录:")
print(f"   {DRIVE_COMFYUI_PATH}/models/checkpoints/")
print(f"   {DRIVE_COMFYUI_PATH}/models/loras/")
print(f"   等等...\n")

## 🌐 步骤 6: 安装 Cloudflared（用于公网访问）

In [ ]:
import os

if not os.path.exists('/usr/local/bin/cloudflared'):
    print("🌐 安装 Cloudflared...")
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
    !dpkg -i cloudflared-linux-amd64.deb > /dev/null 2>&1
    !rm cloudflared-linux-amd64.deb
    print("✅ Cloudflared 安装完成\n")
else:
    print("✅ Cloudflared 已安装\n")

## 🚀 步骤 7: 启动 ComfyUI

In [ ]:
import subprocess
import threading
import time
import socket

print("="*60)
print("🚀 启动 ComfyUI")
print("="*60)
print()

# 切换到 ComfyUI 目录
%cd {DRIVE_COMFYUI_PATH}

def start_tunnel(port):
    """启动 Cloudflare Tunnel"""
    # 等待 ComfyUI 启动
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
    
    print("\n" + "="*60)
    print("🎉 ComfyUI 已启动！")
    print("="*60)
    print("\n🌐 正在创建公网访问链接...\n")
    
    p = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    
    for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com " in l:
            url = l[l.find("http"):].strip()
            print("="*60)
            print(f"✅ 访问地址: {url}")
            print("="*60)
            print("\n💡 使用说明:")
            print("   1. 复制上面的 URL")
            print("   2. 在新标签页中打开")
            print("   3. 开始使用 ComfyUI！")
            print("\n⚠️  注意: 不要关闭此笔记本，否则 ComfyUI 会停止运行")
            print("\n📊 查看实时日志，请向下滚动...\n")
            print("="*60)
            break

# 启动 Tunnel 线程
threading.Thread(target=start_tunnel, daemon=True, args=(PORT,)).start()

# 显示总耗时
total_time = time.time() - start_time
print(f"⏱️  总启动时间: {total_time:.1f}秒\n")
print("🎮 GPU 模式:", LAUNCH_MODE if LAUNCH_MODE else "标准模式")
print()
print("="*60)
print("📝 ComfyUI 日志输出:")
print("="*60)
print()

# 启动 ComfyUI
!python main.py --listen 0.0.0.0 --port {PORT} {LAUNCH_MODE}

---

## 🛠️ 工具单元格

以下单元格包含一些实用工具，可以按需运行。

### 🔄 更新 ComfyUI 核心

In [ ]:
# 运行此单元格以更新 ComfyUI 到最新版本
%cd {DRIVE_COMFYUI_PATH}
!git pull
print("\n✅ ComfyUI 已更新")

### 🔄 更新所有自定义节点

In [ ]:
# 运行此单元格以更新所有自定义节点
import os

%cd {DRIVE_COMFYUI_PATH}/custom_nodes

for node_dir in os.listdir('.'):
    node_path = node_dir
    git_path = os.path.join(node_path, '.git')
    if os.path.isdir(node_path) and os.path.exists(git_path):
        print(f"🔄 更新 {node_dir}...")
        !git -C {node_path} pull
        
        # 重新安装依赖
        req_file = os.path.join(node_path, 'requirements.txt')
        if os.path.exists(req_file):
            !pip install -q -r {req_file}

print("\n✅ 所有节点已更新")

### 📊 查看环境信息

In [ ]:
# 运行此单元格以查看当前环境信息
print("📊 环境信息")
print("="*60)

# GPU
print("\n🎮 GPU:")
!nvidia-smi --query-gpu=name,memory.total,memory.used,memory.free --format=csv

# Python
print("\n🐍 Python:")
!python --version

# PyTorch
print("\n🔥 PyTorch:")
import torch
print(f"   版本: {torch.__version__}")
print(f"   CUDA: {torch.version.cuda}")
print(f"   cuDNN: {torch.backends.cudnn.version()}")
print(f"   可用: {torch.cuda.is_available()}")

# 磁盘
print("\n💾 磁盘使用:")
!df -h {DRIVE_COMFYUI_PATH}

# 自定义节点
print("\n🔌 自定义节点:")
import os
nodes_dir = f"{DRIVE_COMFYUI_PATH}/custom_nodes"
if os.path.exists(nodes_dir):
    nodes = [d for d in os.listdir(nodes_dir) if os.path.isdir(os.path.join(nodes_dir, d))]
    for node in nodes:
        print(f"   ✓ {node}")
    print(f"\n   总计: {len(nodes)} 个节点")
else:
    print("   未找到自定义节点")

print("\n" + "="*60)

### 🗑️ 清理并重新部署

In [ ]:
# ⚠️ 警告: 此单元格会删除现有的 ComfyUI 环境
# 只在需要完全重新部署时运行

import shutil
import os

if os.path.exists(DRIVE_COMFYUI_PATH):
    print("⚠️  即将删除现有环境...")
    print(f"   路径: {DRIVE_COMFYUI_PATH}")
    
    # 备份 models 目录（可选）
    models_path = f"{DRIVE_COMFYUI_PATH}/models"
    if os.path.exists(models_path):
        print("\n💾 检测到模型目录，是否备份？")
        print("   (模型文件可能很大，备份会花费时间)")
        # 在实际使用时可以添加交互确认
    
    print("\n🗑️  删除中...")
    shutil.rmtree(DRIVE_COMFYUI_PATH)
    print("✅ 已删除")
    print("\n💡 重新运行上面的单元格以重新部署")
else:
    print("ℹ️  ComfyUI 环境不存在，无需清理")